<a href="https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset


In [2]:
df = pd.read_json("D:\pan20-authorship-verification-training-small\pan20-authorship-verification-training-small.jsonl", lines = True)
df_labels = pd.read_json("D:\pan20-authorship-verification-training-small\pan20-authorship-verification-training-small-truth.jsonl", lines = True)

In [3]:
df.tail()
#df['pair'][0][0]

,id,fandoms,pair
52596,3926a1bd-6d33-5513-a694-12d7590443d4,"[True Blood, Austin & Ally]",[Enjoy! And time stands still beneath the air ...
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,"[True Blood, Kuroko no Basuke/黒子のバスケ]","[""I forgive you,"" she blurted out looking stra..."
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,"[Austin & Ally, Five Nights at Freddy´s]","[""What? Why? What""s happening? Where are we go..."
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,"[Five Nights at Freddy´s, Austin & Ally]","[""Gah!"" I growled. ""I need a password to acces..."
52600,900c4a19-e22b-59fd-9cc7-8490919b1696,"[Five Nights at Freddy´s, Austin & Ally]",[Chica heard a noise below her and saw Golden ...


We take first 2,000 and last 2,000 sentences from the dataset to ensure uniform distribution of labels

In [4]:
batch_1 = pd.concat([df[:2000], df[-2000:]])
#batch_1.reset_index(drop=True, inplace=True)
batch_1_labels = pd.concat([df_labels[:2000], df_labels[-2000:]])
#batch_1_labels.reset_index(drop=True, inplace=True)
batch_1_labels['same'] = batch_1_labels['same'].astype(int) 

In [4]:
"""
for p, pair in enumerate(batch_1['pair']):
    #for i, text in enumerate(pair):
    #    pair[i] = text[0:250]
    batch_1['pair'][p] = ' [SEP] '.join(pair)
"""
# code for splitting before tokenization

We check how many pairs are labeled as `same` (value 1) and how many are labeled `different` (having the value 0)

In [5]:
batch_1_labels['same'].value_counts()

1    2000
0    2000
Name: same, dtype: int64

## Loading the Pre-trained BERT model

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)

cuda:0


In [7]:
# DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Avoid unnecessary warnings
ppb.logging.set_verbosity_error()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The variable `model` holds a pretrained BERT model.

## Model #1: Preprocessing and Tokenization

Tokenization is performed using BertTokenizer and may take 7-10 minutes

In [8]:
tokenized = batch_1['pair'].apply((lambda x: tokenizer(text = x[0], text_pair = x[1],
add_special_tokens=True, truncation=True, max_length=40, return_tensors="pt").to(device)))

In [9]:
tokenized[0] #let's check what the tokenized input looks like

{'input_ids': tensor([[  101,  1045,  5670,  1037,  2978,  1010, 23625,  5599,  2026,  2159,
         14957,  2013,  2028, 13547,  2000,  1996,  2060,  1011,  1011,  2021,
           102,  1000,  2035,  2097,  2468,  2028,  2007,  3607,  1010,  1000,
          2002,  2056,  1010,  2471,  3432,  1010,  2010, 15138, 18823,   102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [172]:
#tokenized.index = np.arange(0, len(tokenized))
#tokenized = tokenized.reset_index(drop=True, inplace=True)

#re-indexing was required for some experiments, but not amymore. Use re-indexing when you iterate over enumerated rows. 

#tokenized_index[1999]
#tokenized_index[2000]

In [129]:
"""
for i, pair in enumerate(tokenized):
    #input_id = tokenized[pair]['input_ids']
    #tokenized[i] = ' [SEP] '.join(pair['input_ids'])
    #if i == 0:
    #    print(i, "PPAAAAIR", pair)
    #    print("000", tokenized[i]['input_ids'])
    #print(i)
    texts = tokenized[i]['input_ids'][0] + tokenized[i]['input_ids'][1]
    types = tokenized[i]['token_type_ids'][0] + tokenized[i]['token_type_ids'][1]
    masks = tokenized[i]['attention_mask'][0] + tokenized[i]['attention_mask'][1]
    if i > 1999:
        #print(text)
        tokenized[i]['input_ids'] = texts
        tokenized[i]['token_type_ids'] = types
        tokenized[i]['attention_mask'] = masks
"""        

# code for splitting after tokenization. Not used in batched processing.\

"""
token_tensors = []
for i, pair in enumerate(tokenized):
    token_tensors.append(pair['input_ids'])

token_tensors_df = pd.DataFrame(token_tensors)
"""

# code for extracting the encodings. Not used in batched processing.

In [ ]:
"""
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
"""
# code for padding when it is not performed during tokenization

# to simulate padding:
# padded = np.array([i for i in token_tensors_df.values])

#np.array(padded).shape

In [ ]:
"""
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
"""

# adding attention mask if it is not performed during tokenization

## Model #1 Learning

The `model()` function runs our sentences through BERT (it is inherited by BERT's `forward()` function). The results of the processing will be returned into `all_output`

In [10]:
#print(torch.cuda.memory_summary(device=None, abbreviated=False))
torch.cuda.empty_cache()

In [ ]:
"""
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

print(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model = model.to(device)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
"""

# standard training with separate, manually created encodings and attention masks (no batching => may require a lot of memory)

In [220]:
# test model to try different modes 

test_input = tokenizer(["Hello, my dog is cute", "And what now?"], padding=True, return_tensors="pt").to("cuda:0")
type(test_input)
model = model.to("cuda:0")
test_output = model(**test_input)

In [11]:
model = model.to(device)


all_outputs = torch.Tensor()
all_outputs = all_outputs.to(device)

with torch.no_grad():
    for batch in tokenized:
        
        #batch['input_ids'].to(device)              # not needed if CUDA was enabled during tokenization
        #batch['token_type_ids'].to(device)
        #batch['attention_mask'].to(device)

        outputs = model(**batch)
        all_outputs = torch.cat((all_outputs, outputs.last_hidden_state), 0)
    print(all_outputs.shape)       

torch.Size([4000, 40, 768])


In [31]:
#model = model.to(device)

#with torch.no_grad():
#    features = tokenized.apply(lambda x: model(**x).last_hidden_state)


Extract `[CLS]` tokens to used them as input to simple LogReg classifier and save them in the `features` variable

In [39]:
features = all_outputs[:,0,:].cpu().numpy() # [:,0,:] meaning: ':' for all sequences, '0' for first token in sequence, ':' for all 768 hidden layers

The labels indicating which pair is written by the same author or by different ones go into the `labels` variable

We also randomly permute labels and then permute features according to the same pattern. This is done to provide uniform distribution of lables after train/test split

In [40]:
labels = batch_1_labels['same']

idx = np.random.permutation(labels.index)
labels = labels.reindex(idx)
features = pd.DataFrame(features)
features = features.reindex(idx)

## Model #2: Train/Test Split
We now split the obtained datset into a training set and testing set in standard proportion (75% to 25%)

In [52]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [45]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 100.0}
best scores:  0.6076666666666667


In [53]:
lr_clf = LogisticRegression(C=97)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=97)

In [54]:
lr_clf.score(test_features, test_labels)

0.603

In [55]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.503 (+/- 0.00)
